In [159]:
import pandas as pd
from collections import Counter
from scipy import spatial
import math
import re

In [160]:
Codalab = pd.read_csv('train_all_tasks.csv')
Exist2021 = pd.read_csv('EXIST2021_merged.csv')

In [161]:
data1 = Codalab.loc[Codalab['label_sexist'] == 'sexist']
data2 = Exist2021.loc[Exist2021['sexist'] == 1]
data2_nonsexist = Exist2021.loc[Exist2021['sexist'] == 0]

In [165]:
d1_text = []
for n1 in data1['text']:
  res1 = re.findall( r'\w+|[^\s\w]+', n1)
  d1_text.append(res1)

d2_text = []
for n2 in data2['text']:
  res2 = re.findall( r'\w+|[^\s\w]+', n2)
  d2_text.append(res2)

In [191]:
def cosine_sim(t1, t2, label_cat):
  wordlist = list(t1.keys() | t2.keys())
  t1_vec = []
  t2_vec = []
  for word in wordlist:
    t1_vec.append(t1.get(word, 0)) 
    t2_vec.append(t2.get(word, 0))    
  return 1 - spatial.distance.cosine(t1_vec, t2_vec), label_cat

In [192]:
print(len(d1_text))
print(len(d2_text))

3398
2794


In [200]:
def best_score(scores, sent):
  for score, label in sorted(zip(scores, sent), reverse=True)[:1]:
    return label

In [201]:
cos_scores = []
cos_sent = []
Exist2021_label_categories = []
for n in d2_text:
  for n1, s1 in zip(d1_text, data1['label_category']):
    cs, lc = cosine_sim(Counter(n), Counter(n1), s1)
    cos_scores.append(cs)
    cos_sent.append(lc)

  Exist2021_label_categories.append(best_score(cos_scores, cos_sent))
  cos_scores = []
  cos_sent = []

In [ ]:
for x, y in zip(Exist2021_label_categories, d2_text):
  print(x, ' '.join(y))

In [ ]:
data2['label_category'] = Exist2021_label_categories
data2_nonsexist['label_category'] = 'none'

In [214]:
Exist2021_taskB = pd.concat([data2, data2_nonsexist], axis=0)
Exist2021_taskB = Exist2021_taskB.sample(frac=1, random_state=1234)

In [215]:
Codalab['label_category'].value_counts()

none                                        10602
2. derogation                                1590
3. animosity                                 1165
4. prejudiced discussions                     333
1. threats, plans to harm and incitement      310
Name: label_category, dtype: int64

In [216]:
Exist2021_taskB['label_category'].value_counts()

none                                        2850
2. derogation                               1237
3. animosity                                 972
4. prejudiced discussions                    319
1. threats, plans to harm and incitement     266
Name: label_category, dtype: int64

In [217]:
Exist2021_taskB.to_csv('Exist2021_taskB.csv', index=False, header=True)